In [21]:
from mclass import *
from maze import *
from minHeapClass import *
import numpy as np
import random

## MP 1.1 - A_Star Search

This section of the MP is designed to solve 1.1 using the A_Star search algorithm. The heuristic that is used to find the path is the Manhattan Distance between two given points on the maze.

In [22]:
# Method: ManhattanDistance
# Inputs: a, b -> Cells in the maze
# Outputs: The Manhattan Distance between two points in the maze
# Description: This method is used to determine the heuristic in the A* function
def ManhattanDistance(a, b):
    return abs(a.x - b.x) + abs(a.y - b.y)

# Method: Neighbors
# Inputs: maze -> The list of the MazeCells
#         current -> Current cell we want to find neighbors of
# Outputs: children -> List of neighbors that are valid moves
# Description: This method is used to determine the valid neighbors for any given cell in the maze
def neighbors(maze, current):
    children = []
    if not maze[current.left].isWall:
        children.append(maze[current.left])
    if not maze[current.right].isWall:
        children.append(maze[current.right])
    if not maze[current.down].isWall:
        children.append(maze[current.down])
    if not maze[current.up].isWall:
        children.append(maze[current.up])
    return children

# Method: A_Star_Search
# Inputs: maze -> The list of the MazeCells
#         startIdx -> The starting index of the search
#         goalIdx -> The goal index for the search
# Outputs: gVal -> The path costs for traveling between nodes
#          parents -> The dictionary that contains the path
#          count -> The number of expanded nodes
# Description: This method conducts the A* search between the start and goal. It counts the cost of the path as well as the number of expanded nodes in the process
def A_Star_Search(maze, startIdx, goalIdx):
    # Initialize open list, close list, and path list
    openList = MinHeap()
    gVal = {}
    parents = {}
    count = 0
    
    # Push the starting index onto the frontier
    openList.push(maze[startIdx], 0)
    gVal[maze[startIdx]] = 0
    parents[maze[startIdx]] = None
    
    # Keep repeating the process until the frontier is empty
    while not openList.isEmpty():
        # Pop off frontier and increment number of expanded nodes
        current = openList.pop()
        count += 1
        
        # If goal index is found break out of loop
        if current.idx == goalIdx:
            break
        
        # Find successors of current nodes
        children = neighbors(maze, current)
        for child in children:
            # Calculate new cost of going to a certain child in a given direction
            newGVal = gVal[current] + 1
            if child not in gVal or newGVal < gVal[child]:
                # Either update previous cell or add new cell to the frontier
                gVal[child] = newGVal
                parents[child] = current
                fn = newGVal + ManhattanDistance(child, maze[goalIdx])
                openList.push(child, fn)
    
    return gVal, parents, count

# Method: drawPath
# Inputs: maze -> The list of the textfile
#         startIdx -> The starting index of the search
#         goalIdx -> The goal index for the search
#         cellList -> The list of the mazeCells
#         parents -> The path of the traversal
# Outputs: mazeString -> The path in text format
# Description: This method draws the path
def drawPath(maze, cellList, parents, startIdx, goalIdx):
    retPath = []
    current = cellList[goalIdx]
    
    # Add path starting from the goal
    while current != cellList[startIdx]:
        retPath.append(current)
        current = parents[current]
    
    # initialize new maze with the text
    newMaze = list(maze)

    # Add periods to show the path
    for movement in retPath:
        if movement.idx == startIdx:
            continue
        if movement.idx == goalIdx:
            continue
        newMaze[movement.idx] = '.'

    maze_string = ''.join(newMaze)
    print maze_string
    
    return maze_string

The main code below calls the upon the methods described above. We start by opening the file, running the search, and outputting the results into a new text file. All the output files will be saved in the a_star folder in the MP1 directory.

In [23]:
# Retrieve and print unsolved maze
print "Unsolved Maze"
maze, cellList, startIdx, goalIdx = getMaze('/inputMazes/openMaze.txt')
print maze + '\n'

# Conduct A* Search on the maze and draw the solution
print "Solved Maze"
gVals, parents, expanded = A_Star_Search(cellList, startIdx, goalIdx)
solution = drawPath(maze, cellList, parents, startIdx, goalIdx)

# Print the path cost and the number of nodes expanded
print "Cost of Path:", gVals[cellList[goalIdx]]
print "Nodes Expanded:", expanded

# Output the solution to a new text file
f = open('a_star/a_star_open.txt', 'w')
f.write(solution)
f.write("\nCost of Path:" + " " + str(gVals[cellList[goalIdx]]))
f.write("\nNodes Expanded:" + " " + str(expanded))
f.close()

Unsolved Maze
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P            %
%                     %             %
%                     %             %
%                     %             %
%                     %             %
%                     %%%%%%%%      %
%                            %      %
%                            %      %
%                            %      %
%                            %      %
%                            %      %
%      %%%%%%%%%%%%%%%%%            %
%      %                            %
%      %                            %
%      %                            %
%      %                            %
%      %%%%                         %
%        .%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Solved Maze
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P            %
%                     %.            %
%                     %.            %
%                     %.            %
%                     %

## MP 1.2 - A_Star Search With Multiple Dots

This section of the MP is designed to solve 1.2 using the A_Star search algorithm from the previous part. The heuristic that is used to find the path is the sum of the edges of the MST generated between different goal states. We start by calculating the A_Star cost from the starting index to each individual goal state. Then we generate the MST for each goal state, with that specific goal state as the root. We sum the edges on this MST and add it to the path cost from the start state. This becomes the determining factor for calculating which order to visit each goal state.

In [24]:
# Method: generateMST
# Inputs: lookup -> Lookup table that holds the weighted graph between goal indices in the maze
#         root -> The node to root the MST at
# Outputs: retVal -> The edges of the MST
# Description: This method is used as the heuristic for 1.2. It generates an MST using Prim's Algorithm, rooted at a node and then returns the edgelist
def generateMST(lookup, root):
    # Use's Prim's Algorithm to generate the MST
    retVal = []
    visited = []
    
    visited.append(root)
    
    while len(visited) != lookup.shape[0]:
        edges = []
        for vertex in visited:
            for adj in range(lookup.shape[0]):
                if adj not in visited and lookup[vertex][adj] != 0:
                    edges.append((vertex, adj))
        #add smallest edge
        smallestEdge = edges[0]
        for edge in edges:
            # Find the smallest edge
            if lookup[edge[0]][edge[1]] < lookup[smallestEdge[0]][smallestEdge[1]]:
                smallestEdge = edge
            # Random tiebreaker if edge weights are the same
            if lookup[edge[0]][edge[1]] == lookup[smallestEdge[0]][smallestEdge[1]]:
                x = random.randint(1,2)
                if x == 1:
                    continue
                else:
                    smallestEdge = edge
        retVal.append(smallestEdge)
        visited.append(smallestEdge[1])
    
    return retVal

# Method: A_Star_Multiple_Dots
# Inputs: cellList -> The list of the mazeCells
#         startIdx -> The starting index of the search
#         goalList -> The list of all the goals in the graph
# Outputs: order -> The order in which each goal state should be accessed
#          expandedNodes -> The number of expanded nodes on the path taken
#          pathCost -> The cost of path taken to get all goal states
# Description: This method conducts the traversal across the maze using the MST heuristic as the determining factor
def A_Star_Multiple_Dots(cellList, startIdx, goalList):
    # Initialize Lists to determine A* path
    gList = []
    hList = []
    fList = []
    initialPipelineExpansion = []
    # Initialize Lookup tables to create weighted graph to create MST
    lookup = np.zeros((len(goalList), len(goalList)))
    expandedLookup = np.zeros((len(goalList), len(goalList)))

    # Find the A* path from the starting index to each of the goal states
    for finIdx in range(len(goalList)):
        gVals, parents, expanded = A_Star_Search(cellList, startIdx, goalList[finIdx])
        gList.append(gVals[cellList[goalList[finIdx]]])
        initialPipelineExpansion.append(expanded)

    # Fill in the lookup table by running A* from each goal state to the other goal states
    for x in range(len(goalList)):
        for y in range(len(goalList)):
            if x == y:
                lookup[x][y] = 0
                continue
            gVals, parents, expanded = A_Star_Search(cellList, goalList[x], goalList[y])
            lookup[x][y] = gVals[cellList[goalList[y]]]
            expandedLookup[x][y] = expanded

    # Generate MST between all the goal states andsum the edges, this will act as the heuristic for the search
    for finIdx in range(len(goalList)):
        mstSum = 0
        edges = generateMST(lookup, finIdx)
        for edge in edges:
            mstSum += lookup[edge[0]][edge[1]]
        hList.append(mstSum)

    # Calculate the f(n) values for each goal state
    for idx in range(len(gList)):
        fList.append(gList[idx] + hList[idx])
    
    # Order the goal states based on the smallest heuristic values, this is the path that we need to take to get the solution
    order = np.argsort(fList)
    
    # Determine the path cost and the number of expanded nodes if the above order is followed
    pathCost = 0
    expandedNodes = 0
    pathCost += gList[order[0]]
    expandedNodes += initialPipelineExpansion[order[0]]
    for idx in range(len(order) - 1):
        pathCost += lookup[order[idx]][order[idx+1]]
        expandedNodes += expandedLookup[order[idx]][order[idx+1]]
        
    return order, expandedNodes, pathCost

# Method: drawMultipleDotsOrder
# Inputs: maze -> The list of the mazeCells
#         goalList -> The list of all the goals in the graph
#         order -> The order in which to visit the goal states
# Outputs: maze_string -> The text solution of 1.2
# Description: This method draws the order of which goal state to visit
def drawMultipleDotsOrder(maze, goalList, order):
    # To show the solution, we use the following ordering
    ordering = list('123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz')
    
    # Create new list for solution
    newMaze = list(maze2)
    
    # Add order to maze
    for idx in range(len(order)):
        newMaze[goalList[order[idx]]] = ordering[idx]

    maze_string = ''.join(newMaze)
    print maze_string
    
    return maze_string

The main code below calls the upon the methods described above. We start by opening the file, running the search, and outputting the results into a new text file. All the output files will be saved in the a_star folder in the MP1 directory.

In [25]:
# Retrieve and print unsolved maze
print "Unsolved Maze"
maze2, cellList2, startIdx2, goalList = getMazeWithFinList('/inputMazes/tinySearch.txt')
print maze2 + '\n'

# Conduct A* Search on the maze and draw the solution
print "Solved Maze"
order, expandedNodes, pathCost = A_Star_Multiple_Dots(cellList2, startIdx2, goalList)
solution = drawMultipleDotsOrder(maze2, goalList, order)

# Print the path cost and the number of nodes expanded
print "Cost of Path:", pathCost
print "Nodes Expanded:", expandedNodes

# Output the solution to a new text file
f = open('a_star/tinySearchMultipleDots.txt', 'w')
f.write(solution)
f.write("\nCost of Path:" + " " + str(pathCost))
f.write("\nNodes Expanded:" + " " + str(expandedNodes))
f.close()

Unsolved Maze
%%%%%%%%%%
%.  %   .%
% %.% %% %
% %   .%.%
% .%P%   %
%.  .  . %
% %%%% %.%
%.    .% %
%%%%%%%%%%

Solved Maze
%%%%%%%%%%
%8  %   B%
% %2% %% %
% %   3%C%
% 4%P%   %
%5  1  6 %
% %%%% %9%
%A    7% %
%%%%%%%%%%
Cost of Path: 75.0
Nodes Expanded: 127.0
